<a href="https://colab.research.google.com/github/mprocz/QAP-GeneticAlgorithm_MAB-UCB/blob/main/QAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code

## Imports


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import numpy as np
import pandas as pd
from typing import List
import numpy.random as npr
from math import ceil, floor, sqrt, log
from random import randint, uniform, seed, sample, shuffle

##Data

In [ ]:
class Data():
    """
    This class stores the instance file data.

    Attributes:
        instanceName (str): The name of the instance.
        size (int): The size of the problem instance.
        flows (List[List[int]]): A matrix representing flow data between facilities.
        distances (List[List[int]]): A matrix representing distances between locations.
    """

    instanceName: str = ""
    size: int = 0
    flows: List[List[int]] = []
    distances: List[List[int]] = []

def open_instance_file(filePath: str) -> List[List[int]]:
    """
    Opens the instance data file from the specified file path.

    Author: Felipe T. Peruci https://github.com/devLIPEr

    Args:
        filePath (str): The path to the instance data file.

    Returns:
        List[List[int]]: The instance data in a 2D list format.

    Raises:
        Exception: If the file cannot be opened or read.

    The instances data file is available at: (https://coral.ise.lehigh.edu/data-sets/qaplib/).
    """

    try:
        with open(filePath, 'r') as f:
            instance = [([int(a) for a in line.split()]) for line in f if len(line.split())]
            return instance
    except Exception as e:
        print("File not found")
        raise e

def get_file_data(filePath: str, name: str="") -> None:
    """
    Separates each data element from the opened file and stores it in the Data class.

    Author: Felipe T. Peruci https://github.com/devLIPEr

    Args:
        filePath (str): The path to the instance data file.
        name (str): The name of the instance (default is an empty string).

    Raises:
        Exception: If there is an error extracting data from the file.
    """

    try:
        fileData = open_instance_file(filePath)

        Data.size = fileData.pop(0)[0]

        flowB = []
        while len(flowB) < (Data.size*Data.size):
            line = fileData.pop(0)
            flowB.extend(line)

        distB = []
        while len(distB) < (Data.size * Data.size):
            line = fileData.pop(0)
            distB.extend(line)

        Data.flows = [flowB[i : i + Data.size] for i in range(0, len(flowB), Data.size)]
        Data.distances = [distB[i : i + Data.size] for i in range(0, len(distB), Data.size)]
        Data.instanceName = name
    except Exception as exc:
        print("Error extracting data. \nError: ", type(exc), '\n', (exc), '\n')

## Result

In [ ]:
def compare_result(filePath: str, best, printInfo: int=1) -> List:
    """
    Compares the obtained results with the best available result for this instance.

    Author: Felipe T. Peruci https://github.com/devLIPEr

    This function compares the results obtained during execution with the best-known result available at:
    (https://coral.ise.lehigh.edu/data-sets/qaplib/).

    Args:
        filePath (str): Path to the file containing the best-known result.
        best (Individual): The best individual found by the algorithm.
        printInfo (int): Controls whether to print additional information (1 to print, 0 to suppress).

    Returns:
        List: A list containing the error value and the best-known cost from the file.
              Returns ['-1', '-1'] if there is an error opening the file.

    Raises:
        Exception: Prints an error message if the file is not found or another error occurs.
    """

    try:
        with open(filePath) as f:
            result = [([int(a) for a in line.replace(',', ' ').split()]) for line in f]
        cost = result[0][1]

        if best.cost >= cost:
            error = best.cost-cost
            if printInfo == 1:
                if error == 0:
                    print('Best possible found.\n')
                else:
                    pass
                    #print(f'Solution is off by {error} from optimal solution.\n')
                return [error, cost]
        else:
            error = abs(best.cost-cost)
            pass
            #print(f'Solution is better by {error} from optimal solution.\n')
            return [-error, cost]
    except Exception as exc:
        print(f'File not found. \n', type(exc), '\n', (exc), 'n')
    return ['-1', '-1']

##Algorithm

In [ ]:
class Individual():
    """
    This class represents a solution/individual for the problem.

    Attributes:
        chromosome (list): The solution representation as a list.
        cost (int): The cost associated with the solution.
        generation (int): The generation in which the solution was created.
    """

    def __init__(self, chromosome: List = [], generation: int = 1, cost: int = None):
        if cost is None:
            self.chromosome: List[int] = self.generateChromosome(chromosome)
            self.cost: int = self.costCalculation()
            self.generation: int = generation
        else:
            self.chromosome: List[int] = []
            self.cost: int = cost
            self.generation: int = generation

    def __sub__(self, other):
        return self.cost - other.cost

    def __lt__(self, other):
        return self.cost < other.cost

    def __gt__(self, other):
        return self.cost > other.cost

    def generateChromosome(self, chromosome: List) -> List:
        """
        Generates a random chromosome or fixes an existing one by removing duplicates and filling missing values.

        Args:
            chromosome (List[int]): The chromosome to be fixed or an empty list to create a new one.

        Returns:
            List[int]: A chromosome list representing a possible solution.
        """

        if (len(chromosome) == Data.size):
            for i in range(Data.size):
                if chromosome.count(i) >= 2:
                    while(chromosome.count(i) != 1):
                        chromosome.remove(i)
            for i in range(Data.size):
                if i not in chromosome:
                    chromosome.append(i)
            return chromosome
        else:
            seed()
            newChromosome = sample(range(Data.size), Data.size)
            return newChromosome

    def costCalculation(self) -> int:
        """
        Calculates the solution cost by evaluating distances and flows for the chromosome.

        Returns:
            int: The total cost of the current chromosome.
        """

        cost = 0
        for i in range(Data.size):
            for j in range(Data.size):
                cost += ((Data.distances[i][j]) * (Data.flows[self.chromosome[i]][self.chromosome[j]]))
        Evaluations.evaluations += 1
        return cost

    def mutation(self) -> None:
        """
        Performs a mutation on the chromosome by swapping two random points and recalculates the solution cost.
        """

        point1 = randint(0, Data.size-1)
        point2 = randint(0, Data.size-1)
        while point1 == point2:
            point2 = randint(0, Data.size-1)
        aux = self.chromosome[point1]
        self.chromosome[point1] = self.chromosome[point2]
        self.chromosome[point2] = aux
        self.cost = self.costCalculation()

In [ ]:
class GeneticAlgorithm():
    """
    This class represents the Genetic Algorithm.

    Attributes:
        nIndividualsPerGeneration (int): The number of individuals in each generation.
        nGenerations (int): The total number of generations to evolve.
        mutationRate (int): The rate of mutation as a percentage.
        elitismRate (float): The proportion of the best individuals to carry over to the next generation.
        nChildren (int): The number of children to produce from each generation.
        tournamentSize (int): The size of the tournament for selection.
        similarity (float): The similarity threshold for selecting individuals.
        operator (List[str]): Names of all genetic operators.
        crossOperator (str): The current crossover operator being used.
        gen (int): The current generation number.
        best (List[Individual]): A list that stores the best individuals found overall.
        bestIndividualPerGeneration (List[Individual]): A list that stores the best individual for each generation.
        currentPopulation (List[Individual]): The current population of individuals.
        operatorTime (List[float]): The time taken for each operator to execute.
        allOperatorCounter (List[int]): A list that counts how many times each operator has been applied.
    """

    def __init__(self,
                 nIndividualsPerGeneration: int = 100,
                 nGeneration: int = 50,
                 mutationRate: int = 5,
                 elitismRate: float = 0.01,
                 nChildrens: int = 1,
                 tournamentSize: int = 20,
                 similarityRate: float = 0.95,
                 evaluationMultiplier: int =100,
                 crossOperator: str = None):

        self.nIndividualsPerGeneration: int = nIndividualsPerGeneration
        self.nGenerations: int = nGeneration
        self.mutationRate: int = mutationRate
        self.elitismRate: float = elitismRate
        self.nChildren: int = nChildrens
        self.tournamentSize: int = tournamentSize
        self.similarity: float = similarityRate
        self.operator: List[str] = ['OPX', 'CX', 'GSX', 'VR']  # Initialize the list of genetic operators used in the algorithm.
        self.crossOperator: str = crossOperator
        self.gen: int = 1
        self.best: List[Individual] = []
        self.bestIndividualPerGeneration: List[Individual] = []
        Evaluations.evaluations: int = 0
        self.currentPopulation: List[Individual] = []
        self.operatorTime: List[float] = []
        self.allOperatorCounter: List[List[int]] = []

        # Initialize a multi-armed bandit (MAB) using the Upper Confidence Bound 1 (UCB1) strategy.
        self.mab = MAB_UCB1(4, self.operator)

    def generatePopulation(self, population: List[Individual] = []) -> None:
        """
        This method generates a population of solutions.

        The method continues to create new individuals until the current population
        reaches the specified number of individuals per generation.

        Args:
            population (List[Individual]): An optional list of existing individuals (default is an empty list).
        """

        while(len(self.currentPopulation) < self.nIndividualsPerGeneration):
            ind = Individual([], self.gen)
            self.currentPopulation.append(ind)

    def tournament(self) -> Individual:
         """
        This method implements the Tournament Selection Operator.

        The method randomly selects a subset of individuals from the current population,
        sorts them based on their cost, and returns the best individual from the tournament.

        Returns:
            Individual: The best individual selected from the tournament.
        """

        seed()
        pop = sample(self.currentPopulation[0:], self.tournamentSize)
        pop.sort(key = lambda x: x.cost)
        return (pop[0])

    def OLDsendReward(self, parents: List[Individual], offspring: Individual) -> None:
        """
        This method sends the reward of the current operator to the MAB algorithm.

        This older version checks if the offspring's cost is less than or equal to
        the cost of either parent and sends a reward accordingly.

        Args:
            parents (List[Individual]): A list of parent individuals.
            offspring (Individual): The offspring individual generated from the parents.

        """
        if (offspring.cost <= parents[0].cost or offspring.cost <= parents[1].cost):
            self.mab.reward(True)
        else:
            self.mab.reward(False)

    def sendReward(self, parents, child, op):
        """
        This method sends the reward of the current operator to the MAB algorithm.

        Author: Felipe T. Peruci https://github.com/devLIPEr

        This method calculates and sends a reward based on the performance of the child individual in relation to
        its parents, depending on the operator used for generating the child.

        Args:
            parents (List[Individual]): A list of parent individuals.
            child (Individual): The child individual generated from the parents.
            op (str): The operator used to generate the child.
        """
        if op == "MPX":
            rewarded = False
            for parent in parents:
                if(not rewarded and child.cost <= parent.cost):
                    self.mab.reward((parent-child)/parent.cost if parent.cost != 0 else (parent-child))
                    # self.mab.reward((parent-child)/child.cost)
                rewarded = True
            if not rewarded:
                self.mab.reward((parents[-1]-child)/parents[-1].cost if parents[-1].cost != 0 else (parents[-1]-child))
                # self.mab.reward((parents[-1]-child)/child.cost)
        else:
            if(child.cost <= parents[0].cost or child.cost <= parents[1].cost):
                pIdx = 0 if child < parents[0] else 1
                self.mab.reward((parents[pIdx]-child)/parents[pIdx].cost if parents[pIdx].cost != 0 else (parents[pIdx]-child))
                # self.mab.reward((parents[pIdx]-child)/child.cost)
            else:
                self.mab.reward((parents[1]-child)/parents[1].cost if parents[1].cost != 0 else (parents[1]-child))
                # self.mab.reward((parents[1]-child)/child.cost)

    def crossover(self, parents: List[Individual], operator: str) -> Individual:
        """
        Implements various Crossover Operators to generate a new individual.

        This method takes two parent individuals and applies the specified crossover operator to produce a child individual.
        The method supports different crossover techniques such as CX (Cycle Crossover), GSX (Greedy Subtour Crossover),
        VR (Voting Recombination Crossover), and OPX (one-point crossover).

        Args:
            parents (List[Individual]): A list containing two parent individuals from which the child is generated.
            operator (str): The crossover operator to be used ('CX', 'GSX', 'VR', or 'OPX').

        Returns:
            Individual: The newly created child individual resulting from the crossover operation.
        """

        child = None
        if operator == 'CX':
            # Cycle Crossover
            # Reference: Misevicius, A., & Kilda, B. (2005). Comparison of crossover operators for the quadratic assignment problem. Information Technology and Control, 34.
            # Description: CX identifies cycles within the chromosomes of two parents. It alternates genes from each parent to construct the child.
            # The process starts by copying genes that are identical in both parents to the child at the same position.
            # The next gene to copy is determined by the position of the current gene in the other parent, ensuring that no gene is copied more than once.

            parent_one = parents[0]
            parent_two = parents[1]
            chrom_length = Data.size

            child_one = [-1 for i in range(chrom_length)]
            child_two = [-1 for i in range(chrom_length)]


            p1_copy = parent_one.chromosome.copy()
            p2_copy = parent_two.chromosome.copy()

            swap = True
            count = 0
            pos = 0

            while True:
                if count > chrom_length:
                    break
                for i in range(chrom_length):
                    if child_one[i] == -1:
                        pos = i
                        break
                if swap:
                    while True:
                        child_one[pos] = parent_one.chromosome[pos]
                        count += 1
                        pos = parent_two.chromosome.index(parent_one.chromosome[pos])
                        if p1_copy[pos] == -1:
                            swap = False
                            break
                        p1_copy[pos] = -1
                else:
                    while True:
                        child_one[pos] = parent_two.chromosome[pos]
                        count += 1
                        pos = parent_one.chromosome.index(parent_two.chromosome[pos])
                        if p2_copy[pos] == -1:
                            swap = True
                            break
                        p2_copy[pos] = -1
            for i in range(chrom_length):
                if child_one[i] == parent_one.chromosome[i]:
                    child_two[i] = parent_two.chromosome[i]
                else:
                    child_two[i] = parent_one.chromosome[i]

            for i in range(chrom_length):
                if child_one[i] == -1:
                    if p1_copy[i] == -1:
                        child_one[i] = parent_two.chromosome[i]
                    else:
                        child_one[i] = parent_one.chromosome[i]

            child = Individual(chromosome=child_one, generation=self.gen)
            #child2 = Individual(chromosome=child_two, generation=self.gen)


        elif operator == 'GSX':
            # Greedy Subtour Crossover
            # Reference: Umbarkar, A. J., & Sheth, P. D. (2015). Crossover operators in genetic algorithms: a review. ICTACT journal on soft computing, 6(1). http://doi.org/10.21917/ijsc.2015.0150
            # Description: GSX aims to pass the longest sequence of subtours from the parents to the child, helping to avoid local minima.
            # It begins by selecting a random gene and finding its position in both parents.
            # The algorithm then alternates between adding genes from the left of the selected gene in one parent and from the right in the other.
            # If a parent runs out of genes or a gene is already present in the child, it stops adding from that parent, filling any gaps randomly.

            seed()
            pivot = randint(0, Data.size-1)

            #print(parents[0].chromosome)
            p1Index = parents[0].chromosome.index(pivot) - 1
            p2Index = parents[1].chromosome.index(pivot) + 1

            newChromosome = []
            newChromosome.append(pivot)
            p1 = True
            if p1Index < 0:
                p1 = False
            p2 = True
            if p2Index >= Data.size:
                p2 = False

            while p1 or p2:
                if p1:
                    if(parents[0].chromosome[p1Index] not in newChromosome):
                        newChromosome.insert(0, parents[0].chromosome[p1Index])
                        p1Index -= 1
                        if p1Index < 0:
                            p1 = False
                    else:
                        p1 = False

                if p2:
                    if(parents[1].chromosome[p2Index] not in newChromosome):
                        newChromosome.append(parents[1].chromosome[p2Index])
                        p2Index += 1

                        if p2Index >= Data.size:
                            p2 = False
                    else:
                        p2 = False

            res = []
            for i in range(Data.size):
                if i not in newChromosome:
                    res.append(i)
            shuffle(res)
            for i in res:
                newChromosome.append(i)

            child = Individual(chromosome=newChromosome, generation=self.gen)

        elif operator == 'VR':
            # Voting Recombination Crossover
            # Reference: Umbarkar, A. J., & Sheth, P. D. (2015). Crossover operators in genetic algorithms: a review. ICTACT journal on soft computing, 6(1). http://doi.org/10.21917/ijsc.2015.0150
            # Description: VR is a crossover operator that can utilize multiple parents (at least two) to generate a child.
            # It checks each gene's position in the chromosome against the other parents.
            # If the gene exists in the other parents, it is copied to the child. Remaining genes are filled in randomly at the end of the child’s chromosome.

            offspring = [-1 for i in range(Data.size)]
            for i in range(Data.size):
                if parents[0].chromosome[i] == parents[1].chromosome[i]:
                    offspring[i] = parents[0].chromosome[i]

            for i in range(Data.size):
                if i not in offspring:
                    seed()
                    pos = randint(0, Data.size-1)
                    while(offspring[pos] != -1):
                        seed()
                        pos = randint(0, Data.size-1)
                    offspring[pos] = i
            child = Individual(chromosome=offspring, generation=self.gen)

        elif operator == 'OPX':
            # One-Point Crossover
            # Reference: Misevicius, A., & Kilda, B. (2005). Comparison of crossover operators for the quadratic assignment problem. Information Technology and Control, 34.
            # Description: OPX is a straightforward crossover operator that selects a cutting point in a chromosome.
            # Genes from one parent are copied from the start up to the cutting point, followed by genes from the other parent from the cutting point to the end of the chromosome.

            p = randint(0, Data.size-1)
            newChromosome = parents[0].chromosome[:p] + parents[1].chromosome[p:]
            child = Individual(chromosome=newChromosome, generation=self.gen)

        self.sendReward(parents, child, operator)

        seed()
        if randint(1, 100) < self.mutationRate:
                child.mutation()

        return child

    def generationsLoop(self) -> None:
        """
        This method represents the main loop of the Genetic Algorithm. It generates the initial population,
        evaluates individuals, performs selection, crossover, mutation, and updates the population for a specified
        number of generations. It also implements elitism, similarity verification, and tracks the best individuals.
        """

        self.generatePopulation()
        self.currentPopulation.sort(key = lambda x: x.cost)
        self.best.append(self.currentPopulation[0])

        while (self.gen < self.nGenerations):
            ### operator counter list
            operatorCounter = [0,0,0,0]

            ### generation counter
            self.gen += 1

            ### individuals sorting
            self.currentPopulation.sort(key = lambda x: x.cost)

            ### similarity verification
            similarity = 0
            for ind in self.currentPopulation:
                similarity += ind.cost
            similarity /= self.nIndividualsPerGeneration
            if similarity >= self.similarity:
                self.currentPopulation = self.currentPopulation[:ceil(self.nIndividualsPerGeneration/4)]
                self.generatePopulation(self.currentPopulation)

            ### elitism selection
            newPopulation = []
            for i in range(floor(self.nIndividualsPerGeneration*self.elitismRate)):
                newPopulation.append(self.currentPopulation[i])

            ### tournament selection and crossovers
            offs = len(newPopulation)
            while (offs < self.nIndividualsPerGeneration):
                self.crossOperator = self.mab.pull()
                operatorCounter[self.operator.index(self.crossOperator)] += 1
                ind1 = self.tournament()
                ind2 = self.tournament()
                for i in range(self.nChildren):
                    initialTime = timer()
                    offspring = self.crossover([ind1, ind2], self.crossOperator)
                    self.operatorTime.append(timer() - initialTime)
                    newPopulation.append(offspring)
                    offs +=  1

            ### individuals sorting
            newPopulation.sort(key = lambda x: x.cost)

            ### population size verification
            if (len(self.currentPopulation) > self.nIndividualsPerGeneration):
                self.currentPopulation = newPopulation[:self.nIndividualsPerGeneration]

            ### best individual verification
            if self.currentPopulation[0].cost < self.best[0].cost:
                self.best[0] = self.currentPopulation[0]

            self.allOperatorCounter.append(operatorCounter.copy())

In [ ]:
class MAB_UCB1():
    """
    This class implements the Multi-Armed Bandit (MAB) Upper Confidence Bound (UCB1) Algorithm
    as a Hyper-Heuristic for a Genetic Algorithm.

    Attributes:
        numbersOfOperators (int): Number of available operators for selection.
        operators (List[str]): List of operator names corresponding to the indices.
        trials (int): Total number of trials (pulls) performed.
        armsPulled (List[int]): Count of times each operator has been pulled.
        cumulativeRewards (List[float]): Cumulative rewards received for each operator.
        averageRewards (List[float]): Average reward calculated for each operator.
        currentReward (List[float]): Reward values for the current selection of operators.
        currentChosenArm (int | None): The index of the currently chosen operator.
    """

    def __init__(self,
                 numbersOfOperators: int,
                 operators: List[str]):
        #constantes
        self.numbersOfOperators: int = numbersOfOperators
        self.operators: List[str] = operators
        self.rewardMult = 100
        #variáveis
        self.trials: int = 1
        self.armsPulled: List[int] = [0 for i in range(numbersOfOperators)]
        self.cumulativeRewards: List[int] = [0 for i in range(numbersOfOperators)]
        self.averageRewards: List[int] = [0 for i in range(numbersOfOperators)]
        self.currentReward: List[int] = [0 for i in range(numbersOfOperators)]
        self.currentChosenArm: int | None = None

    def updateValues(self) -> None:
        """
        Updates the cumulative and average rewards for each operator based on the current rewards.

        This method is called after receiving a reward for the currently chosen operator.
        """
        for i in range(self.numbersOfOperators):
            self.cumulativeRewards[i] += self.currentReward[i]
            self.averageRewards[i] = ( self.cumulativeRewards[i] / self.trials )


    def pull(self, returnStr: bool = True) -> int | str:
        """
        Chooses an operator based on the UCB1 algorithm and returns its index or name.

        This method checks if there are any untried operators. If all operators have been pulled at least once,
        it calculates the UCB1 values and selects the operator with the highest value.

        Args:
            returnStr (bool): If True, returns the operator's name; otherwise, returns the index.

        Returns:
            int | str: The index of the chosen operator or its name based on the returnStr flag.

        Reference: Auer, P. (2003). Using confidence bounds for exploitation-exploration trade-offs . Machine learning, 3, 397-422.
        """

        # Check if there is a current chosen arm; if so, prompt for a reward first
        if self.currentChosenArm is not None:
            return "First, reward the last arm"

        # Iterate through all operators to find any that have not been pulled yet
        for i in range(self.numbersOfOperators):
            if self.armsPulled[i] == 0:
                self.currentChosenArm = i
                self.armsPulled[i] += 1
                if returnStr: return self.operators[self.currentChosenArm]
                else: return self.currentReward.index(self.currentChosenArm)

        # If all operators have been pulled at least once, calculate UCB values for each operator
        for i, operator in enumerate(self.operators):
            self.currentReward[i] = (self.averageRewards[i] + (sqrt((2 * log(self.trials)) / self.armsPulled[i])))

        # Select the operator with the maximum UCB value
        self.currentChosenArm = self.currentReward.index(max(self.currentReward))

        self.trials += 1
        self.armsPulled[self.currentChosenArm] += 1

        if returnStr: return self.operators[self.currentChosenArm]
        else: return self.currentChosenArm

    def reward(self, rewardValue: int) -> None:
        """
        Updates the rewards of the currently chosen operator based on the received reward value.

        Args:
            rewardValue (int): The reward value to be added to the cumulative reward of the current operator.

        Raises:
            Exception: If no operator has been pulled before calling this method.
        """

        try:
            self.cumulativeRewards[self.currentChosenArm] += rewardValue * self.rewardMult
        except:
            print("First, pull an arm")
        self.updateValues() # Update cumulative and average rewards
        self.currentChosenArm = None  # Reset the chosen arm

In [ ]:
nIndividualsPerGeneration = 100
nGenerations = 100
crossoverRate = 95
mutationRate = 5
elitismRate = 0.01
nChildrens = 1
tournamentSize = 20
similarity = 0.95

get_file_data(os.path.join(file_diretory_path, fileName), file_name)
ga = GeneticAlgorithm(nIndividualsPerGeneration,
                      nGenerations,
                      crossoverRate,
                      mutationRate,
                      elitismRate,
                      nChildrens,
                      tournamentSize,
                      similarity)
ga.generationsLoop()